# Running 10bp dtrim through methlykit

In [24]:
wd="/Volumes/Serine/wd/18-04-29"

In [25]:
mkdir $wd

In [27]:
cd $wd

/Volumes/Serine/wd/18-04-29


In [28]:
%%bash
find /Volumes/Serine/wd/18-04-27/zr2096_*R1* \
| xargs basename -s _s1_R1_val_1.fq.gz | xargs -I{} /Applications/bioinfo/Bismark_v0.19.0/bismark \
--path_to_bowtie /Applications/bioinfo/bowtie2-2.3.4.1-macos-x86_64 \
--genome /Volumes/Serine/wd/18-03-15/genome \
--score_min L,0,-1.2 \
-p 4 \
--non_directional \
-1 /Volumes/Serine/wd/18-04-27/{}_s1_R1_val_1.fq.gz \
-2 /Volumes/Serine/wd/18-04-27/{}_s1_R2_val_2.fq.gz \
2> bismark.err



FastQ format assumed (by default)
Each Bowtie 2 instance is going to be run with 4 threads. Please monitor performance closely and tune down if needed!
chr NC_035780.1 (65668440 bp)
chr NC_035781.1 (61752955 bp)
chr NC_035782.1 (77061148 bp)
chr NC_035783.1 (59691872 bp)
chr NC_035784.1 (98698416 bp)
chr NC_035785.1 (51258098 bp)
chr NC_035786.1 (57830854 bp)
chr NC_035787.1 (75944018 bp)
chr NC_035788.1 (104168038 bp)
chr NC_035789.1 (32650045 bp)
chr NC_007175.2 (17244 bp)

Number of paired-end alignments with a unique best hit:	9282636
Mapping efficiency:	53.2%

Sequence pairs with no alignments under any condition:	4524985
Sequence pairs did not map uniquely:	3641262
Sequence pairs which were discarded because genomic sequence could not be extracted:	0

Number of sequence pairs with unique best (first) alignment came from the bowtie output:
CT/GA/CT:	2365261	((converted) top strand)
GA/CT/CT:	2269368	(complementary to (converted) top strand)
GA/CT/GA:	2275104	(complementary to (con

In [29]:
%%bash

/Applications/bioinfo/Bismark_v0.19.0/deduplicate_bismark \
--bam -p \
*.bam \
2> dedup.err



Now testing Bismark result file zr2096_10_s1_R1_val_1_bismark_bt2_pe.bam for positional sorting (which would be bad...)	skipping header line:	@HD	VN:1.0	SO:unsorted
skipping header line:	@SQ	SN:NC_035780.1	LN:65668440
skipping header line:	@SQ	SN:NC_035781.1	LN:61752955
skipping header line:	@SQ	SN:NC_035782.1	LN:77061148
skipping header line:	@SQ	SN:NC_035783.1	LN:59691872
skipping header line:	@SQ	SN:NC_035784.1	LN:98698416
skipping header line:	@SQ	SN:NC_035785.1	LN:51258098
skipping header line:	@SQ	SN:NC_035786.1	LN:57830854
skipping header line:	@SQ	SN:NC_035787.1	LN:75944018
skipping header line:	@SQ	SN:NC_035788.1	LN:104168038
skipping header line:	@SQ	SN:NC_035789.1	LN:32650045
skipping header line:	@SQ	SN:NC_007175.2	LN:17244
skipping header line:	@PG	ID:Bismark	VN:v0.19.0	CL:"bismark --path_to_bowtie /Applications/bioinfo/bowtie2-2.3.4.1-macos-x86_64 --genome /Volumes/Serine/wd/18-03-15/genome --score_min L,0,-1.2 -p 4 --non_directional -1 /Volumes/Serine/wd/18-04-27/zr2096

In [30]:
%%bash
find *deduplicated.bam \
| xargs basename -s _s1_R1_val_1_bismark_bt2_pe.deduplicated.bam | xargs -I{} /Applications/bioinfo/samtools-1.3.1/samtools \
sort {}_s1_R1_val_1_bismark_bt2_pe.deduplicated.bam -o {}_dedup.sorted.bam


[bam_sort_core] merging from 13 files...
[bam_sort_core] merging from 11 files...
[bam_sort_core] merging from 21 files...
[bam_sort_core] merging from 22 files...
[bam_sort_core] merging from 19 files...
[bam_sort_core] merging from 23 files...
[bam_sort_core] merging from 19 files...
[bam_sort_core] merging from 22 files...
[bam_sort_core] merging from 20 files...
[bam_sort_core] merging from 21 files...


```
library(methylKit)

file.list_a=list('zr2096_1_dedup.sorted.bam',
 'zr2096_2_dedup.sorted.bam',
  'zr2096_3_dedup.sorted.bam',
  'zr2096_4_dedup.sorted.bam',
  'zr2096_5_dedup.sorted.bam',
  'zr2096_6_dedup.sorted.bam',
  'zr2096_6_dedup.sorted.bam',
  'zr2096_8_dedup.sorted.bam',
  'zr2096_9_dedup.sorted.bam',
  'zr2096_10_dedup.sorted.bam'
  )



myobj2 = processBismarkAln(location = file.list_a, sample.id = list("1","2","3","4","5","6","7","8","9","10"), assembly = "v3", read.context="CpG", mincov=1, treatment = c(0,0,0,0,0,1,1,1,1,1))



getMethylationStats(myobj2[[1]],plot=FALSE,both.strands=FALSE)

getMethylationStats(myobj2[[1]],plot=TRUE,both.strands=FALSE)

getCoverageStats(myobj2[[4]],plot=TRUE,both.strands=FALSE)




meth=unite(myobj2)



getCorrelation(meth,plot=TRUE)


clusterSamples(meth, dist="correlation", method="ward", plot=TRUE)

```

In [31]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark_methylation_extractor \
-p --bedGraph --counts --scaffolds \
--multicore 8 \
*deduplicated.bam \
2> bme.err


Now testing Bismark result file zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.bam for positional sorting (which would be bad...)	zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.1
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.2
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.3
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.4
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.5
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.6
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.7
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.8

zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.1.mbias
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.2.mbias
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt.3.mbias
zr2096_10_s1_R1_val_1_bismark_bt2_pe.dedupli

In [32]:
!gunzip *bedGraph.gz

In [33]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark2report

Found 10 alignment reports in current directory. Now trying to figure out whether there are corresponding optional reports

Writing Bismark HTML report to >> zr2096_10_s1_R1_val_1_bismark_bt2_PE_report.html <<

Using the following alignment report:		> zr2096_10_s1_R1_val_1_bismark_bt2_PE_report.txt <
Processing alignment report zr2096_10_s1_R1_val_1_bismark_bt2_PE_report.txt ...
Complete

Using the following deduplication report:	> zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplication_report.txt <
Processing deduplication report zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplication_report.txt ...
Complete

Using the following splitting report:		> zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt <
Processing splitting report zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated_splitting_report.txt ...
Complete

Using the following M-bias report:		> zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.M-bias.txt <
Processing M-bias report zr2096_10_s1_R1_val_1_bismark_bt2_pe.de

In [34]:
!#open *report.html

In [35]:
%%bash
/Applications/bioinfo/Bismark_v0.19.0/bismark2summary

No Bismark/Bowtie2 single-end BAM files detected
Found Bismark/Bowtie2 paired-end files
No Bismark/Bowtie single-end BAM files detected
No Bismark/Bowtie paired-end BAM files detected

Generating Bismark summary report from 10 Bismark BAM file(s)...
>> Reading from Bismark report: zr2096_10_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_1_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_2_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_3_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_4_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_5_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_6_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_7_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096_8_s1_R1_val_1_bismark_bt2_PE_report.txt
>> Reading from Bismark report: zr2096